In [ ]:
#hide
from dash_oop_components.core import *

# Deploying with gunicorn

> instructions on how to deploy a dashboard with gunicorn



## Example

An example dashboard can be found at [github.com/oegedijk/dash_oop_demo](https://github.com/oegedijk/dash_oop_demo) and has been deployed to [https://dash-oop-demo.herokuapp.com/](https://dash-oop-demo.herokuapp.com/)

First define the `DashFigureFactory` and `DashComponents` in `dashboard.components.py`. In this case I named them `CovidPlots` and `CovidDashboard`.

Then build the dashboard and save to `dashboard.yaml`:

**build_dashboard.py**:
```python
from dashboard_components import CovidPlots, CovidDashboard
from dash_bootstrap_components.themes import FLATLY
from dash_oop_components import DashApp

plot_factory = CovidPlots(datafile="covid.csv")
dashboard_component = CovidDashboard(plot_factory)
db = DashApp(dashboard_component, external_stylesheets=[FLATLY])
db.to_yaml("dashboard.yaml")
```

Then define a `dashboard.py` that builds the dashboard from config, and exposes the Flask server:

**dashboard.py**
```python
from dash_oop_components import DashApp

db = DashApp.from_yaml("dashboard.yaml")
app = db.app.server
```

And start the gunicorn server:

```bash
$ gunicorn --preload -b localhost:8050 dashboard:app
```

## Automatically reloading dashboard whenever config changes

You can also automatically restart the gunicorn server whenever there is a change to `dashboard.yaml`, by using `watchdog`. Install with `pip install watchdog[watchmedo]`. Start the gunicorn server while saving it's pid:

```bash
$ gunicorn --pid gunicorn.pid --preload -b localhost:8050 dashboard:app
```

And the start a watchmedo script that runs `kill -HUP` on the gunicorn server in order to force a restart whenever it detects a change to `dashboard.yaml`:

```bash
$ watchmedo shell-command -p "./dashboard.yaml" -c 'kill -HUP $(cat gunicorn.pid)'
```

## Loading with pickled `DashFigureFactory`

In some cases you might be doing some expensive calculations inside your `DashFigureFactory` that you do not want to run everytime you restart a dashboard as it would break the gunicorn timeout window.

**build_dashboard.py**:
```python
from dashboard_components import CovidPlots, CovidDashboard
from dash_bootstrap_components.themes import FLATLY
from dash_oop_components import DashApp

plot_factory = CovidPlots(datafile="covid.csv")
dashboard_component = CovidDashboard(plot_factory)
db = DashApp(dashboard_component, external_stylesheets=[FLATLY])

plot_factory.dump("plot_factory.pkl")
plot_factory.to_yaml('plot_factory.yaml')
dashboard.to_yaml("dash_component.yaml")
db.to_yaml("dashboard.yaml")
```

Now you can load the plot_factory from disk. And you can use the fact that you can override parameters when loading with `.from_yaml()` by passing them as kwargs:

**dashboard.py:**
```python
from dash_oop_components import DashFigureFactory, DashComponent, DashApp

plot_factory = DashFigureFactory.from_file("plot_factory.pkl")
dashboard_component = DashComponent.from_yaml("dash_component.yaml", plot_factory=plot_factory)
db = DashApp.from_yaml("dashboard.yaml", dashboard_component=dashboard_component)
app = db.app.server
```

## Fully automated build and redeploy cycle

We can also you watchmedo to rebuild the `plot_factory` whenever there is a change to `plot_factory.yaml` or `covid.csv`,
and then rebuild the dashboard whenever there is a change to `plot_factory.pkl`, `dashboard_component.yaml` or `dashboard.yaml`:

**build_plot_factory.py**:
```python
from dash_oop_component import DashFigureFactory

plot_factory = DashFigureFactory.from_yaml("plot_factory.yaml")
plot_factory.dump("plot_factory.pkl")
```

**build_dashboard.py**:
```python
from dashboard_components import CovidPlots, CovidDashboard
from dash_bootstrap_components.themes import FLATLY
from dash_oop_components import DashApp

plot_factory = DashFigureFactory.from_file("plot_factory.pkl")
dashboard_component = CovidDashboard(plot_factory)
db = DashApp(dashboard_component, external_stylesheets=[FLATLY])

dashboard.to_yaml("dash_component.yaml")
db.to_yaml("dashboard.yaml")
```

**dashboard.py:**
```python
from dash_oop_components import DashFigureFactory, DashComponent, DashApp

plot_factory = DashFigureFactory.from_file("plot_factory.pkl")
dashboard_component = DashComponent.from_yaml("dash_component.yaml", plot_factory=plot_factory)
db = DashApp.from_yaml("dashboard.yaml", dashboard_component=dashboard_component)
app = db.app.server
```

Now start the gunicorn server with:

```bash
$ gunicorn --pid gunicorn.pid --preload -b localhost:8050 dashboard:app
```

Now run the `build_plot_factory.py` script everytime you detect a change in either `covid.csv` or `plot_factory.yaml`:

```bash
$ watchmedo shell-command -p "./covid.csv;./plot_factory.yaml" -c 'python build_plot_factory.py'
```

And restart the gunicorn server everytime you detect a change to `plot_factory.pkl`, `dashboard_component.yaml` or `dashboard.yaml`:

```bash
$ watchmedo shell-command -p "./plot_factory.pkl;./dashboard_component.yaml;./dashboard.yaml" -c 'kill -HUP $(cat gunicorn.pid)'
```